In [1]:
Project = '/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Projects/Adipocyte'
from IPython.display import display
import pandas as pd
import gseapy as gp
import numpy as np
import pickle

gp.__version__

'1.0.5'

In [2]:
conv = pd.read_csv('/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/GSEA/chip/Mouse_Ensembl_Gene_ID_MSigDB.v2022.1.Mm.chip', sep='\t')
conv = conv.drop(columns='Gene Title').rename(columns={'Probe Set ID':'gene_id', 'Gene Symbol':'external_gene_name'})
display(conv)

,gene_id,external_gene_name
0,ENSMUSG00000000001,Gnai3
1,ENSMUSG00000000003,Pbsn
2,ENSMUSG00000000028,Cdc45
3,ENSMUSG00000000031,H19
4,ENSMUSG00000000037,Scml2
...,...,...
43242,ENSMUSG00002076818,SNORD14
43243,ENSMUSG00002076971,Snord100
43244,ENSMUSG00002076975,Gm23823
43245,ENSMUSG00002076989,Gm23510


In [4]:
BG_genes = pd.read_csv(f'{Project}/Results/Plots/Ribosome_stalling_effect_on_protein/v20230917/BG_genes.txt', names=['gene_id'])
BG_Symbols = pd.merge(BG_genes, conv, on='gene_id')
BG_Symbols = BG_Symbols.to_dict('list')['external_gene_name']

genelists = {'Glu-rich-stalled' : pd.read_csv(f'{Project}/Results/Plots/Ribosome_stalling_effect_on_protein/v20230917/Glu-rich-stalled_genes.txt', names=['gene_id'])}

display(BG_Symbols[:10])
display(genelists)

['Gnai3',
 'Narf',
 'Klf6',
 'Cox5a',
 'Fer',
 'Tfe3',
 'Dlat',
 'Sdhd',
 'Ccnd2',
 'Gpr107']

{'Glu-rich-stalled':                 gene_id
 0    ENSMUSG00000020463
 1    ENSMUSG00000053774
 2    ENSMUSG00000071533
 3    ENSMUSG00000063273
 4    ENSMUSG00000004451
 ..                  ...
 201  ENSMUSG00000031799
 202  ENSMUSG00000028464
 203  ENSMUSG00000027940
 204  ENSMUSG00000032249
 205  ENSMUSG00000030122
 
 [206 rows x 1 columns]}

In [5]:
# EnrichR
for group in genelists:
    gene_symbols = pd.merge(genelists[group], conv, on='gene_id')[['external_gene_name']]

    enr = gp.enrichr(   gene_list=gene_symbols,
                        gene_sets='/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/GSEA/gmt/v20221025/m5.go.v2022.1.Mm.symbols.level3to5.gmt', # ['KEGG_2016']
                        background=BG_Symbols,
                        outdir=None, 
                        verbose=False
                    )
    tmp = enr.results.sort_values('Adjusted P-value')
    tmp.to_csv(f'{group}_Enrich.tsv', sep='\t', index=False)
    display(tmp[:20])

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score,Genes
1759,m5.go.v2022.1.Mm.symbols.level3to5.gmt,GOMF_CYTOSKELETAL_PROTEIN_BINDING,41/286,5.340389e-13,1.061135e-09,4.124090,116.539793,Tpm3-rs7;Epb41l3;Pacsin2;Calm1;Tpm2;Arpc4;Calm...
1707,m5.go.v2022.1.Mm.symbols.level3to5.gmt,GOMF_ACTIN_BINDING,24/117,2.996328e-11,2.976852e-08,5.574644,135.079467,Tpm3-rs7;Epb41l3;Tpm2;Arpc4;Bloc1s6;Tpm3;P4hb;...
229,m5.go.v2022.1.Mm.symbols.level3to5.gmt,GOBP_CYTOSKELETON_ORGANIZATION,42/394,4.554994e-09,3.016924e-06,3.014829,57.905952,Tpm3-rs7;Epb41l3;Pacsin2;Tpm2;Arpc4;Vim;Ehbp1l...
1349,m5.go.v2022.1.Mm.symbols.level3to5.gmt,GOBP_SUPRAMOLECULAR_FIBER_ORGANIZATION,29/256,4.402304e-07,2.186844e-04,3.071379,44.952608,Tpm3-rs7;Tpm2;Arpc4;Vim;Tpm3;Myh9;Rdx;Cald1;Tm...
1984,m5.go.v2022.1.Mm.symbols.level3to5.gmt,GOMF_UNFOLDED_PROTEIN_BINDING,13/63,1.159159e-06,3.899482e-04,5.418775,74.062816,Hsp90b1;Hsp90ab1;Npm1;Pet100;Pfdn6;Naca;Cdc37;...
168,m5.go.v2022.1.Mm.symbols.level3to5.gmt,GOBP_CELL_MORPHOGENESIS,32/314,1.177498e-06,3.899482e-04,2.772870,37.855554,Cfdp1;Pacsin2;Vim;P4hb;Aplp2;Myh9;Snx1;Rdx;Arh...
1735,m5.go.v2022.1.Mm.symbols.level3to5.gmt,GOMF_CALCIUM_ION_BINDING,20/147,1.866959e-06,5.299496e-04,3.613185,47.662241,Nucb1;Calm1;Myl6;Calm3;S100a4;Ncl;Sparc;Sdf4;T...
1708,m5.go.v2022.1.Mm.symbols.level3to5.gmt,GOMF_ACTIN_FILAMENT_BINDING,12/61,5.155764e-06,1.280563e-03,5.155879,62.774866,Tpm3-rs7;Marcks;Tpm2;Arpc4;Sptbn1;Bloc1s6;Tpm3...
1448,m5.go.v2022.1.Mm.symbols.level3to5.gmt,GOCC_CELL_CORTEX,16/109,7.814581e-06,1.725286e-03,3.864929,45.449711,Marcks;Epb41l2;Sptbn1;Tpm3;Septin11;Myh9;Rdx;F...
1615,m5.go.v2022.1.Mm.symbols.level3to5.gmt,GOCC_POLYMERIC_CYTOSKELETAL_FIBER,23/209,1.244352e-05,2.247752e-03,2.925995,33.047093,Tpm3-rs7;Calm1;Tpm2;Vim;Ehbp1l1;Tpm3;Calm3;Chm...
